In [6]:
# OFFICIAL COCO EVALUATION
from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances

from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

'''
REAL COCO EVALUATION
'''
def evaluation(annFile,resFile,outFile = "results.txt"):
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    
    cocoGt=COCO(annFile)
    cocoDt=cocoGt.loadRes(resFile)
    imgIds=sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
    cocoEval.params.imgIds  = imgIds
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    
    
def main(ann_file, pred_file):
    
     evaluation(ann_file,pred_file)

print('OUR FILE')        
ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
net_predictions_file = './out.json'

main(ann_file_path, net_predictions_file)

print('\n\n')

print('THEIR FILE')        
ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
net_predictions_file = './their_small_dataset.json'

main(ann_file_path, net_predictions_file)


OUR FILE
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.23s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.105
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.024
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.046
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.291
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.050
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.098
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all 

In [7]:
# NIGHTOWL EVALUATION

'''
Function that takes in input the ground truth file of the annotation 
and the output of a network in the json format and outputs the
miss rates in the output file
'''
def evaluation(annFile,resFile,outFile = "results.txt"):
    from coco import COCO # IMPORT THEIR COCO, not pycocotools
    from eval_MR_multisetup import COCOeval
    
    # running evaluation
    res_file = open("results.txt", "w")
    for id_setup in range(0,4):
        cocoGt = COCO(annFile)
        cocoDt = cocoGt.loadRes(resFile)
        imgIds = sorted(cocoGt.getImgIds())
        cocoEval = COCOeval(cocoGt,cocoDt,'bbox')
        cocoEval.params.imgIds  = imgIds
        cocoEval.evaluate(id_setup)
        cocoEval.accumulate()
        cocoEval.summarize(id_setup,res_file)

    res_file.close()

def main(ann_file, pred_file):
    
     evaluation(ann_file,pred_file)

print('OUR FILE')        
ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
net_predictions_file = './out.json'

main(ann_file_path, net_predictions_file)

print('\n\n')
print('THEIR FILE')        
ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
net_predictions_file = './their_small_dataset.json'

main(ann_file_path, net_predictions_file)



OUR FILE
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[50:10000000000] | visibility=[0.65:10000000000.00] ] = 55.83%
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Miss Rate  (MR) @ Reasonable_small   [ IoU=0.50      | height=[50:75] | visibility=[0.65:10000000000.00] ] = 63.35%
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating in

In [ ]:
# PLOTTING

from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances

from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image

from our_dataset_onlyped import OurDataset

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import json

%matplotlib inline

def main(ann_file_path,images_dir,net_predictions_file):
    # train set
    train_set = OurDataset()
    train_set.load_dataset(images_dir, ann_file_path, is_train=True, val_percentage = 0.0)
    train_set.prepare()
    print('Train images: %d' % len(train_set.image_ids))
    
    input_file = open (net_predictions_file)
    datastore = json.load(input_file)
    
    for image_id in train_set.image_ids:
        info = train_set.image_info[image_id]
        real_id = info['real_id']
        #print(info)
        if image_id < 30 and image_id > 25:
            continue
        # load the image
        image = train_set.load_image(image_id)
        # load the masks and the class ids
        mask, class_ids = train_set.load_mask(image_id)
        # extract bounding boxes from the masks
        bbox = extract_bboxes(mask)
        # display image with masks and bounding boxes
        display_instances(image, bbox, mask, class_ids, train_set.class_names)
        
        for i in datastore:
            if i['image_id'] == real_id:
                '''
                if i['score'] < 0.0001 :
                    continue
                '''
                
                im = np.array(Image.open(info['path']), dtype=np.uint8)
                bbox = i['bbox']
                
                # Create figure and axes
                fig,ax = plt.subplots(1)

                # Display the image
                ax.imshow(im)

                # Create a Rectangle patch
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')

                # Add the patch to the Axes
                ax.add_patch(rect)

                plt.show()
                
        

    
ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
net_predictions_file = './their_very_small_dataset.json'
    
main(ann_file_path,images_path,net_predictions_file)